In [ ]:
import constants
import yfinance as yf

# Get symbols from constants file
lines = constants.sa_str.splitlines()
symbols = [line.split("\t")[1] for line in lines][:100]

# Download data
df = yf.download(symbols + ["SPY"], period="1y", interval="1h", ignore_tz=True)

In [ ]:
import data_pipes

hanzo_data = data_pipes.hanzo_df_array(symbols, df)
hanzo_data['y'].std()

In [43]:
import torch
from torch import nn, optim

class hanzo_0(nn.Module):
    def __init__(self, hist):
        super().__init__()
        self.flat0 = nn.Flatten()
        self.hidden0 = nn.Linear(hist*2, hist)
        self.act0 = nn.Tanh()
        self.hidden1 = nn.Linear(hist, hist)
        self.act1 = nn.Tanh()
        self.hidden2 = nn.Linear(hist, 1)
        self.act2 = nn.Tanh()
 
    def forward(self, x):
        x = self.flat0(x)
        x = self.act0(self.hidden0(x))
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        return x
    

class hanzo_1(nn.Module):
    def __init__(self, hist, dropout_rate=0.3):
        super().__init__()
        self.flat0 = nn.Flatten(start_dim=1)
        self.hidden0 = nn.Linear(hist * 2, hist)
        self.act0 = nn.ReLU()
        self.dropout0 = nn.Dropout(dropout_rate)
        
        self.hidden1 = nn.Linear(hist, hist)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.hidden2 = nn.Linear(hist, 1)
        self.act2 = nn.Tanh()  # Keeping tanh for the output since it's likely regression

    def forward(self, x):
        x = self.flat0(x)
        x = self.dropout0(self.act0(self.hidden0(x)))
        x = self.dropout1(self.act1(self.hidden1(x)))
        x = self.act2(self.hidden2(x))
        return x

In [58]:
import numpy as np
import torch_ops as ops
from copy import deepcopy

hist = 10
epochs, batch_size, lr, l2_decay = 200, 32, 1e-4, 1e-5
model = hanzo_1(hist=hist)
loss_fn = nn.L1Loss()
optimizer = optim.Adam(params=model.parameters(), lr=lr, weight_decay=l2_decay)

period = 70
x = torch.tensor(hanzo_data["x"], dtype=torch.float32)
y = torch.tensor(hanzo_data["y"], dtype=torch.float32)

In [70]:
t = period
# for t in range(period, len(x)):
x_t = x[t - period:t]
x_v = x[t - 1:t]
x_t_flat = torch.concatenate([x_t[:, i] for i in range(x_t.shape[1])])
x_v_flat = torch.concatenate([x_v[:, i] for i in range(x_v.shape[1])])

y_t = y[t - period:t]
y_v = y[t - 1:t]
y_t_flat = torch.concatenate([y_t[:, i] for i in range(y_t.shape[1])])
y_v_flat = torch.concatenate([y_v[:, i] for i in range(y_v.shape[1])])

test_loss = 999
for epoch in range(epochs):
    loss = ops.train_loop(x_t_flat, y_t_flat, model, loss_fn, optimizer, batch_size)
    test_loss_new = ops.test_loop(x_v_flat, y_v_flat, model, loss_fn, batch_size)

    if test_loss_new < test_loss or epoch == epochs - 1:
        print(f"---------- Epoch {epoch + 1} ----------")
        print(f"loss: {loss:1.4f}, test loss: {test_loss_new:1.4f}")
        
    if test_loss_new < test_loss:
        test_loss = test_loss_new
        print(f"Checkpoint: test loss = {test_loss:1.4f} <---------------")
        state_dict_save = deepcopy(model.state_dict())
        torch.save(model.state_dict(), r"models/hanzo_0")

---------- Epoch 1 ----------
loss: 0.1018, test loss: 0.3153
Checkpoint: test loss = 0.3153 <---------------
---------- Epoch 2 ----------
loss: 0.0896, test loss: 0.3153
Checkpoint: test loss = 0.3153 <---------------
---------- Epoch 3 ----------
loss: 0.0872, test loss: 0.3153
Checkpoint: test loss = 0.3153 <---------------
---------- Epoch 4 ----------
loss: 0.1046, test loss: 0.3153
Checkpoint: test loss = 0.3153 <---------------
---------- Epoch 71 ----------
loss: 0.0889, test loss: 0.3153
Checkpoint: test loss = 0.3153 <---------------
---------- Epoch 72 ----------
loss: 0.0950, test loss: 0.3153
Checkpoint: test loss = 0.3153 <---------------
---------- Epoch 73 ----------
loss: 0.0879, test loss: 0.3152
Checkpoint: test loss = 0.3152 <---------------
---------- Epoch 74 ----------
loss: 0.1026, test loss: 0.3152
Checkpoint: test loss = 0.3152 <---------------
---------- Epoch 75 ----------
loss: 0.1002, test loss: 0.3152
Checkpoint: test loss = 0.3152 <---------------
-----

In [34]:
t = period
x_t = x[t - period:t]
x_v = x[t - 1:t]
x_t_flat = torch.concatenate([x_t[:, i] for i in range(x_t.shape[1])])
x_v_flat = torch.concatenate([x_v[:, i] for i in range(x_v.shape[1])])

y_t = y[t - period:t]
y_v = y[t - 1:t]
y_t_flat = torch.concatenate([y_t[:, i] for i in range(y_t.shape[1])])
y_v_flat = torch.concatenate([y_v[:, i] for i in range(y_v.shape[1])])
print(x_t_flat.shape)

torch.Size([7070, 10, 2])


In [36]:
x_t_flat.shape

torch.Size([7070, 10, 2])